![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install geonamescache

In [ ]:
import re
from datetime import datetime, timedelta

import geonamescache
gc = geonamescache.GeonamesCache()

from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import nltk

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
# DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
# df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
# df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
# sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')
df_train = pd.read_csv('main_task.csv')
df_test = pd.read_csv('kaggle_task.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

df = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
df.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
df.sample(5)

In [ ]:
df.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

###  Обработка признаков


In [ ]:
df.isna().sum().plot(kind='bar',title='Количество пропусков')

# Признаки Cuisine Style, Price Range, Number of Reviews  содержат много пропусков.
# Заполним их и создадим новые признаки,"is_NAN" , которые будут говорить о наличии пропуска)

In [ ]:
df['Cuisine Style_NAN'] = df['Cuisine Style'].isna().astype('uint8')
df['Price Range_NAN'] = df['Price Range'].isna().astype('uint8')
df['Number of Reviews_NAN'] = df['Number of Reviews'].isna().astype('uint8')

# **Restaurant_id**

In [ ]:
df.loc[:,['Restaurant_id']]

# Признак не содержит полезной информации, удалим его

In [ ]:
df = df.drop(['Restaurant_id'],axis=1)

# City

In [ ]:
def get_population(city):
    total_info = gc.get_cities_by_name(city)
    if total_info != []:
        total_info = gc.get_cities_by_name(city)[0]
        city_code = next(iter(total_info.keys()))
        population = total_info[city_code]['population']
        return population

    else:
        if city == 'Oporto':
            population = 214349
        elif city == 'Zurich':
            population = 402762
        elif city == 'Krakow':
            population = 779115
        return population

In [ ]:
country = {
    'Paris': 'France',
    'Hamburg': 'Germany',
    'Rome': 'Italy',
    'London': 'UK',
    'Milan': 'Italy',
    'Madrid': 'Spain',
    'Oslo': 'Norway',
    'Stockholm': 'Sweden',
    'Krakow': 'Poland',
    'Lyon': 'Paris',
    'Lisbon': 'Portugal',
    'Edinburgh': 'UK',
    'Vienna': 'Austria',
    'Warsaw': 'Poland',
    'Amsterdam': 'Netherlands',
    'Budapest': 'Hungary',
    'Helsinki': 'Finland',
    'Zurich': 'Switzerland',
    'Luxembourg': 'Luxembourg',
    'Berlin': 'Germany',
    'Prague': 'Czechia',
    'Munich': 'Germany',
    'Bratislava': 'Slovakia',
    'Brussels': 'Belgium',
    'Ljubljana': 'Slovenia',
    'Copenhagen': 'Denmark',
    'Oporto': 'Portugal',
    'Barcelona': 'Spain',
    'Geneva': 'Switzerland',
    'Athens': 'Greece',
    'Dublin': 'Ireland'
}

In [ ]:
plt.figure(figsize=(12,5))
df.City.value_counts().plot(kind='bar', title='Количество ресторанов в городах')

In [ ]:
city_count = dict(df['City'].value_counts())
df['rest_count'] = df['City'].map(city_count)

In [ ]:
df['population'] = df['City'].apply(lambda x: get_population(x))

In [ ]:
df['country'] = df['City'].map(country)

In [ ]:
df['rest_per_people'] = df['rest_count'] / df['population']

In [ ]:
capital_list = ['London', 'Paris', 'Stockholm', 'Madrid', 'Berlin',
                'Rome', 'Prague', 'Lisbon', 'Vienna', 'Amsterdam', 
                'Budapest', 'Warsaw', 'Dublin', 'Copenhagen',
                'Athens', 'Edinburgh', 'Oslo', 'Helsinki', 
                'Bratislava', 'Ljubljana', 'Brussels', 'Luxembourg']

df['is_capital'] = df['City'].apply(lambda x: 1 if x in capital_list else 0)

In [ ]:
df.columns

# Вывод 
### На практике признаки "столица или нет" и "отношение кол-ва ресторанов к кол-ву людей в городе" результат не дали. В итоговом решении учтены не будут.

# Cuisine Style

In [ ]:
# Получаем кол-во редких кухонь
def get_rare_cuisines_count(cuis):
    x = 0
    for i in rare_cuisins_list:
        if i in cuis:
            x += 1
    return x

In [ ]:
# Получаем кол-во часто встречающихся кухонь
def get_common_cuisines_count(cuis):
    x = 0
    for i in common_cuisins_list:
        if i in cuis:
            x += 1
    return x

In [ ]:
def fill_ones(x):
    if cus in x:
        return 1
    return 0

In [ ]:
def is_rare(cuis):
    for i in cuis:
        if i in rare_cuisins_list:
            return 1
        else:
            return 0
    
    
    
def is_common(cuis):
    for i in cuis:
        if i in common_cuisins_list:
            return 1
        else:
            return 0

In [ ]:
# Заполняем пустые значения "Other"
df['Cuisine Style'].fillna("['Other']", inplace=True)

In [ ]:
# Преобразуем строчные значения в списки
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: eval(x))

In [ ]:
# Количество кухонь в ресторане
df['count_cuisines'] = df['Cuisine Style'].apply(lambda x: len(x))

In [ ]:
plt.figure(figsize=(20, 6))
pd.Series.explode(df['Cuisine Style']).value_counts().plot(
    kind='bar', title='Колличество встреч кухонь по видам')

### Создадим два признака "is_rare", "rare_cuisines_count" со значениями 1 и 0 в первом, в зависимости от того, есть ли среди кухонь ресторана редкие или часто встречающиеся, и с количеством редких кухонь во втором. За критерий редкости возьмем медиальное значение этого параметра.

In [ ]:
# Взорвали столбец "Cuisine Style"
exploded_cuisin_list = pd.Series.explode(df['Cuisine Style']).value_counts()

In [ ]:
# Список кухонь, количество встреч которых, мы наблюдаем реже чем медиальное значение общего количества встреч кухонь.
rare_cuisins_list = exploded_cuisin_list.loc[exploded_cuisin_list < round(
    exploded_cuisin_list.median())].index.tolist()

In [ ]:
# Список кухонь, количество встреч которых, мы наблюдаем чаще чем медиальное значение общего количества встреч кухонь.
common_cuisins_list = exploded_cuisin_list.loc[exploded_cuisin_list > round(
    exploded_cuisin_list.median())].index.tolist()

In [ ]:
df['rare_cuisines_count'] = df['Cuisine Style'].apply(get_rare_cuisines_count)
    

In [ ]:
# Признаки с часто встречающимися кухнями результат не дали
# df['common_cuisines_count'] = df['Cuisine Style'].apply(get_common_cuisines_count)
# df['is_common'] = df['Cuisine Style'].apply(is_common)

In [ ]:
df['is_rare'] = df['Cuisine Style'].apply(is_rare)

In [ ]:
cuisines = pd.Series.explode(df['Cuisine Style']).unique()

# Ranking

In [ ]:
plt.rcParams['figure.figsize'] = (15,6)
df.Ranking.hist(bins=150)

# Price Range

In [ ]:
df.isna().sum()

In [ ]:
df['Price Range'].value_counts(dropna=False)

In [ ]:
price_dict = {'$': 1,
              '$$ - $$$': 2,
              '$$$$': 3}

In [ ]:
df['Price Range'] = df['Price Range'].map(price_dict)

# Заполним пропуски наиболее часто встречающимся значением 2

In [ ]:
df['Price Range'].fillna(2, inplace=True)

# Number of Reviews

In [ ]:
df['Number of Reviews'].isna().sum()

In [ ]:
df['Number of Reviews'].fillna(0, inplace=True)

In [ ]:
df['Number of Reviews'].value_counts()

# Reviews

Столбец строковый, содержит информацию с отзывами и датами отзывов. Много незаполненных значений формата "[[], []]". 

In [ ]:
def get_date(s: 'str'):
    fin = []
    tmp = re.findall('\d\d/\d\d/\d{4}', s)
    if [] not in tmp:
        for i in tmp:
            dt_tmp = datetime.strptime(i, '%m/%d/%Y')
            fin.append(dt_tmp)

        return fin
    else:
        return [0, 0]

In [ ]:
def get_delta_date(x):    
    if len(x) > 1:
        return abs((x[0]-x[1]).days)
    else:
        return abs((x[0]-x[0]).days)

In [ ]:
df['Reviews'].fillna('[[], []]',inplace=True)

In [ ]:
df['date_review'] = df['Reviews'].apply(lambda x: get_date(x))

In [ ]:
# Найдем дату первого отзыва по всем ресторанам и заполним ей пустые значения
dates = []
for i in df.date_review:
    if i != None:
        for j in i:
            dates.append(j)
min_date = min(dates)

In [ ]:
df['date_review'] = df['date_review'].apply(lambda x: [min_date,min_date] if x ==[] else x)

In [ ]:
df['date_review'].apply(lambda x: [min_date,min_date] if x ==[] else x)

In [ ]:
# Берем первый элемент из списка дат в каждой строке, 
# так как он соответствует более позднему комментарию
df['last_review_date'] = df['date_review'].apply(lambda x: x[0])

In [ ]:
# Берем первый элемент из списка дат в каждой строке, 
# если количество дат отзывов меньше двух. Иначе второй,так как может быть 3 даты отзывов
df['prelast_review_date'] = df['date_review'].apply(
    lambda x: x[0] if len(x) < 2 else x[1])

In [ ]:
df.sample(5)

Признак с количеством дней между последними отзывами "delta_date"

In [ ]:
df['delta_date'] = df['date_review'].apply(lambda x: get_delta_date(x))

In [ ]:
df['delta_date'].isna().sum()

Признак с количеством дней между последним оставленным отзывом и сегодняшней датой

In [ ]:
today = datetime.now()
df['delta_current_date'] = df['last_review_date'].apply(lambda x: (today - x).days)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(df.drop(['sample'], axis=1).corr(),)

## Изучив таблицу коррелиции признаков, убираем сильно коррелириющие признаки "is_common", "common_cuisines_count", 

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Dummies
### Добавим Dummies в конце, для удобства отслеживания эффективности добавленных признаков

In [ ]:
dummy_city = df.City.str.get_dummies()
df = pd.concat([df,dummy_city],axis=1)


In [ ]:
for cus in cuisines:
    df[cus] = df['Cuisine Style'].apply(fill_ones)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv('main_task.csv')
df_test = pd.read_csv('kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''

    df_output = df_input.copy()

    # ################### 0. Функции ##############################################################
    # Получаем информацию из сторонней библиотеки
    def get_population(city):
        total_info = gc.get_cities_by_name(city)
        if total_info != []:
            total_info = gc.get_cities_by_name(city)[0]
            city_code = next(iter(total_info.keys()))
            population = total_info[city_code]['population']
            return population
        else:
            if city == 'Oporto':
                population = 214349
            elif city == 'Zurich':
                population = 402762
            elif city == 'Krakow':
                population = 779115
            return population

    # Получаем кол-во редких кухонь
    def get_rare_cuisines_count(cuis):
        x = 0
        for i in rare_cuisins_list:
            if i in cuis:
                x += 1
        return x
    
    

    # Функция энкодинга кухонь
    def fill_ones(x):
        if cus in x:
            return 1
        return 0
    
    
    
    
    # Функция присваивающая 1, если кухня среди редких, иначе- 0
    def is_rare(cuis):
        for i in cuis:
            if i in rare_cuisins_list:
                return 1
            else:
                return 0
            
            

    # Получаем дату из отзывов
    def get_date(s: 'str'):
        fin = []
        tmp = re.findall('\d\d/\d\d/\d{4}', s)
        if [] not in tmp:
            for i in tmp:
                dt_tmp = datetime.strptime(i, '%m/%d/%Y')
                fin.append(dt_tmp)
            return fin
        else:
            return [0, 0]

    # Получаем разницу между отзывами в днях
    def get_delta_date(x):
        if len(x) > 1:
            return abs((x[0]-x[1]).days)
        else:
            return abs((x[0]-x[0]).days)
        
        
    # Полярность отзыва
    def polarity_rev(x):
        if '[]' not in x:
            text = x
            sent = TextBlob(text)
            polarity = sent.sentiment.polarity
            return polarity
        else:
            return None

    # ################### 1. Предобработка ##############################################################
    # убираем ненужные для модели признаки. 'URL_TA' убрали, так как парсить я не умею
    df_output.drop(['Restaurant_id', 'ID_TA', 'URL_TA'], axis=1, inplace=True)
    df_output['Cuisine Style_NAN'] = df_output['Cuisine Style'].isna().astype(
        'uint8')
    df_output['Price Range_NAN'] = df_output['Price Range'].isna().astype('uint8')
    
    # Список кухонь
    cities = df_output['City'].unique()
    
    # Словарь город:кол-во ресторанов
    city_count = dict(df_output['City'].value_counts())
    
    # Словарь город:страна
    country = {
        'Paris': 'France',
        'Hamburg': 'Germany',
        'Rome': 'Italy',
        'London': 'UK',
        'Milan': 'Italy',
        'Madrid': 'Spain',
        'Oslo': 'Norway',
        'Stockholm': 'Sweden',
        'Krakow': 'Poland',
        'Lyon': 'Paris',
        'Lisbon': 'Portugal',
        'Edinburgh': 'UK',
        'Vienna': 'Austria',
        'Warsaw': 'Poland',
        'Amsterdam': 'Netherlands',
        'Budapest': 'Hungary',
        'Helsinki': 'Finland',
        'Zurich': 'Switzerland',
        'Luxembourg': 'Luxembourg',
        'Berlin': 'Germany',
        'Prague': 'Czechia',
        'Munich': 'Germany',
        'Bratislava': 'Slovakia',
        'Brussels': 'Belgium',
        'Ljubljana': 'Slovenia',
        'Copenhagen': 'Denmark',
        'Oporto': 'Portugal',
        'Barcelona': 'Spain',
        'Geneva': 'Switzerland',
        'Athens': 'Greece',
        'Dublin': 'Ireland'
    }
    # Список столиц

    # ################### 2. NAN ##############################################################

    df_output['Number of Reviews'].fillna(0, inplace=True)
    df_output['Reviews'].fillna('[[], []]', inplace=True)
    df_output['Cuisine Style'].fillna("['Other']", inplace=True)
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(
        lambda x: eval(x))

    # Взорвали столбец "Cuisine Style"
    exploded_cuisin_list = pd.Series.explode(
        df_output['Cuisine Style']).value_counts()

    # Список кухонь, количество встреч которых, мы наблюдаем реже чем медиальное значение общего количества встреч кухонь.
    rare_cuisins_list = exploded_cuisin_list.loc[exploded_cuisin_list < round(
        exploded_cuisin_list.median())].index.tolist()

    # Список кухонь
    cuisines = pd.Series.explode(df['Cuisine Style']).unique()

    # Словарь ценового диапазона
    price_dict = {'$': 1,
                  '$$ - $$$': 2,
                  '$$$$': 3}

    # Заменяем строковые значения числовыми и заполняем пропуски наиболее часто встреяающимся значением
    df_output['Price Range'] = df_output['Price Range'].map(price_dict)
    df_output['Price Range'].fillna(2, inplace=True)

    # ################### 3. Encoding ##############################################################

    # City
    dummies = df_output.City.str.get_dummies()
    pd.concat([df_output, dummies], axis=1)

    # Cuisins
    for cus in cuisines:
        df_output[cus] = df_output['Cuisine Style'].apply(fill_ones)
    # ################### 4. Feature Engineering ####################################################

    # Количество кухонь в ресторане
    df_output['count_cuisines'] = df_output['Cuisine Style'].apply(
        lambda x: len(x))

    # Количество редких кухонь
    df_output['rare_cuisines_count'] = df_output['Cuisine Style'].apply(
        get_rare_cuisines_count)

    # Бинарный признак редкости кухни
    df_output['is_rare'] = df_output['Cuisine Style'].apply(is_rare)

    # Количество ресторанов в городе
    df_output['rest_count'] = df_output['City'].map(city_count)

    # Количество людей в городе
    df_output['population'] = df_output['City'].apply(
        lambda x: get_population(x))

    # Даты отзывов
    df_output['date_review'] = df_output['Reviews'].apply(
        lambda x: get_date(x))

    # Дата первого отзыва по всем отзывам
    min_date = pd.Series.explode(df_output['date_review']).min()

    # Заполнение пустых значений датой первого отзыва
    df_output['date_review'] = df_output['date_review'].apply(
        lambda x: [min_date, min_date] if x == [] else x)

    # Берем первый элемент из списка дат в каждой строке, так как он соответствует более позднему комментарию
    df_output['last_review_date'] = df_output['date_review'].apply(
        lambda x: x[0])

    # Берем первый элемент из списка дат в каждой строке, если количество дат отзывов меньше двух.
    # Иначе- второй,так как может быть 3 даты отзывов
    df_output['prelast_review_date'] = df_output['date_review'].apply(
        lambda x: x[0] if len(x) < 2 else x[1])

    # Признак количества дней между последними отзывами
    df_output['delta_date'] = df_output['date_review'].apply(
        lambda x: get_delta_date(x))

    # Признак количества дней между последним отзывом и сегодняшней датой
    today = datetime.now()
    df_output['delta_current_date'] = df_output['last_review_date'].apply(
        lambda x: (today - x).days)
        
    #  Признак полярности отзыва
    df_output['polarity_rev'] = df_output.Reviews.apply(lambda x: polarity_rev(x))
    df_output['polarity_rev'] = df_output['polarity_rev'].fillna(df_output.polarity_rev.mean())
    # ################### 5. Clean ####################################################
    # убираем признаки которые еще не успели обработать,
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [
        s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis=1, inplace=True)

    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.drop(['last_review_date','prelast_review_date'], axis=1, inplace=True)

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(df_preproc.corr(),)

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Видим разницу в том, что реальные рейтинги всегда кратны 0.5
# Напишем функцию соответствующей корректировки предсказанных рейтингов
def fine_rating_pred(rating_pred):
    if rating_pred <= 0.5:
        return 0.0
    if rating_pred <= 1.5:
        return 1.0
    if rating_pred <= 1.75:
        return 1.5
    if rating_pred <= 2.25:
        return 2.0
    if rating_pred <= 2.75:
        return 2.5
    if rating_pred <= 3.25:
        return 3.0
    if rating_pred <= 3.75:
        return 3.5
    if rating_pred <= 4.25:
        return 4.0
    if rating_pred <= 4.75:
        return 4.5
    return 5.0

In [ ]:
# Применим такое округление
for i in range(len(y_pred)):
    y_pred[i] = fine_rating_pred(y_pred[i])

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# pre 0.1731875
# best 0.1731875

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
# Применим округление
for i in range(len(predict_submission)):
    predict_submission[i] = fine_rating_pred(predict_submission[i])

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
